In [2]:
import os
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
RAW_DATA_PATH = '../data/raw'
PROCESSED_DATA_PATH = '../data/processed'
PATH_WIKIDATA = os.path.join(RAW_DATA_PATH, 'wikidata')
PATH_WIKIDATA_CHARS = os.path.join(RAW_DATA_PATH, 'wikidata_chars')

In [5]:
df_chars = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, 'wikidata_chars.csv'))
df_chars.head()

,movie_id,director_id,producer_id,box_office_value,cast_id_0,cast_id_1,cast_id_2,cast_id_3,cast_id_4
0,Q1000394,Q1985392,Q179200,NaN,Q40475,Q461370,Q922580,Q15069875,Q2831060
1,Q1000174,Q1379822,NaN,NaN,Q108233,Q95951,Q1578075,Q2061747,Q1442953
2,Q1000825,Q360509,Q716064,NaN,Q2370079,Q1041384,Q4272908,Q13016199,Q16305292
3,Q1001102,Q3199956,NaN,NaN,Q1269500,Q603685,Q1658517,Q1253847,Q12628320
4,Q1000826,Q450601,Q3559718,NaN,Q298818,Q313040,Q945099,Q3751926,Q465643


Create dictionnaries to map QIDs to itemLabels

In [5]:
#Get all the different genders and sex:
url = 'https://query.wikidata.org/sparql'
query = '''
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item WHERE {
      {
        ?item p:P31 ?statement0.
        ?statement0 (ps:P31/(wdt:P279*)) wd:Q290. #sex
      } UNION {
        ?item p:P31 ?statement0.
        ?statement0 (ps:P31/(wdt:P279*)) wd:Q48277. #gender
      }
      UNION {
        ?item p:P31 ?statement0.
        ?statement0 (ps:P31/(wdt:P279*)) wd:Q18382802. #sex or gender
      }
      UNION {
        ?item p:P31 ?statement0.
        ?statement0 (ps:P31/(wdt:P279*)) wd:Q4369513. #sex of humans
      }
    }
  }
}
'''
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

genders_dict = {}
for d in data['results']['bindings']:
    genders_dict[d['item']['value'].split('/')[-1]] = d['itemLabel']['value']

with open(os.path.join(RAW_DATA_PATH, 'genders_dict.json'), 'w') as fp:
    json.dump(genders_dict, fp)


# Get all the countries
url = 'https://query.wikidata.org/sparql'
query = '''
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item WHERE {
        ?item p:P31 ?statement0.
        ?statement0 (ps:P31/(wdt:P279*)) wd:Q6256. #country
    }
  }
}
'''
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

countries_dict = {}
for d in data['results']['bindings']:
    countries_dict[d['item']['value'].split('/')[-1]] = d['itemLabel']['value']

with open(os.path.join(RAW_DATA_PATH, 'countries_dict.json'), 'w') as fp:
    json.dump(countries_dict, fp)


# Get all the genres, might take some time ~20 seconds
url = 'https://query.wikidata.org/sparql'
query = '''
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item WHERE {
        ?item p:P31 ?statement0.
        ?statement0 (ps:P31/(wdt:P279*)) wd:Q483394. #genre
    }
  }
}
'''
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

genres_dict = {}
for d in data['results']['bindings']:
    genres_dict[d['item']['value'].split('/')[-1]] = d['itemLabel']['value']

with open(os.path.join(RAW_DATA_PATH, 'genres_dict.json'), 'w') as fp:
    json.dump(genres_dict, fp)

In [6]:
def get_name(data, id):
    try:
        name = data.get('entities').get(id).get('labels').get('en').get('value')
    except:
        name = np.NaN
    return name
    

def get_gender_id(data, id):
    try:
        gender = data.get('entities').get(id).get('claims').get('P21')[0].get('mainsnak').get('datavalue').get('value').get('id')
    except:
        gender = np.NaN
    return gender

def get_country_id(data, id):
    try:
        country = data.get('entities').get(id).get('claims').get('P27')[0].get('mainsnak').get('datavalue').get('value').get('id')
    except:
        try:
            country = data.get('entities').get(id).get('claims').get('P495')[0].get('mainsnak').get('datavalue').get('value').get('id')
        except:
            try:
                country = data.get('entities').get(id).get('claims').get('P17')[0].get('mainsnak').get('datavalue').get('value').get('id')
            except:
                country = np.NaN

    return country
    
def get_birthdate(data, id):
    try:
        # First try birth date
        date_str = data.get('entities').get(id).get('claims').get('P569')[0].get('mainsnak').get('datavalue').get('value').get('time')[1:11]
    except:
        try:
            # Then try creation date (Inception) 
            date_str = data.get('entities').get(id).get('claims').get('P571')[0].get('mainsnak').get('datavalue').get('value').get('time')[1:11]
        except:
            try:
                # Then try publication date
                date_str = data.get('entities').get(id).get('claims').get('P577')[0].get('mainsnak').get('datavalue').get('value').get('time')[1:11]
            except:
                try:
                    # Last try work period (start)
                    date_str = data.get('entities').get(id).get('claims').get('P2031')[0].get('mainsnak').get('datavalue').get('value').get('time')[1:11]
                except:
                    return np.NaN

    if date_str[-2:] == '00': # Remove days if they are 00
        date_str = date_str[:-3]
    if date_str[-2:] == '00':
        date_str = date_str[:-3] # Remove month if they are 00

    date = np.NaN
    if len(date_str)==4:
        # TODO: Correct way? Changes the date to yyyy-01-01
        # Other option create sep. column for year, month and day. -> not good for readability 
        # Date will be mainly used to get age at movie release. Most actors/directors/producers are adults so the biggest error 
        # of 364 will not have that big of an impact when comparing e.g. a 20 year old with a 60 year old
        try:
            date = datetime.strptime(date_str, '%Y').date() 
        except:
            print('Date {} from ID {} is not in the correct format.'.format(date_str, id))
    elif len(date_str)==7:
        try:
            date = datetime.strptime(date_str, '%Y-%m').date() 
        except:
            print('Date {} from ID {} is not in the correct format.'.format(date_str, id))    
    elif len(date_str)==10:
        try:
            date = datetime.strptime(date_str, '%Y-%m-%d').date()
        except:
            print('Date {} from ID {} is not in the correct format.'.format(date_str, id))
    else:
        print('Date {} from ID {} is not in the correct format.'.format(date_str, id))
    return date


def get_genre_id(data, id):
    try:
        genre = data.get('entities').get(id).get('claims').get('P136')[0].get('mainsnak').get('datavalue').get('value').get('id')
    except:
        genre = np.NaN
    return genre



Extend the entire dataframe

In [15]:
col_ext = ['movie_id', 'director_id', 'producer_id', 'cast_id_0', 'cast_id_1', 'cast_id_2', 'cast_id_3', 'cast_id_4']

def extend_movie(row):  

    for col in col_ext:
        id = row[col]
        if pd.isnull(id):
            continue

        if id == row['movie_id']:
            with open(os.path.join(PATH_WIKIDATA, '{}.json'.format(id))) as data_json:
                data = json.load(data_json)
                row['movie_name'] = get_name(data, id)
                row['movie_genre'] = genres_dict.get(get_genre_id(data, id))
                row['movie_origin'] = countries_dict.get(get_country_id(data, id))
                row['movie_release'] = get_birthdate(data, id)

        else:
            with open(os.path.join(PATH_WIKIDATA_CHARS, '{}/{}.json'.format(row['movie_id'], id))) as data_json:
                data = json.load(data_json)
                row[col + '_name'] = get_name(data, id)
                row[col + '_gender'] = genders_dict.get(get_gender_id(data, id))
                row[col + '_country'] = countries_dict.get(get_country_id(data, id))
                row[col + '_birth'] = get_birthdate(data, id)

            if not pd.isnull(row['movie_release']) and not pd.isnull(row[col + '_birth']):
                row[col + '_age_at_release'] = row['movie_release'].year - row[col + '_birth'].year

    return row

df_chars_extended = df_chars.apply(extend_movie, axis=1)
df_chars_extended.head()

,box_office_value,cast_id_0,cast_id_0_age_at_release,cast_id_0_birth,cast_id_0_country,cast_id_0_gender,cast_id_0_name,cast_id_1,cast_id_1_age_at_release,cast_id_1_birth,...,movie_id,movie_name,movie_origin,movie_release,producer_id,producer_id_age_at_release,producer_id_birth,producer_id_country,producer_id_gender,producer_id_name
0,NaN,Q40475,27.0,1904-03-23,United States of America,female,Joan Crawford,Q461370,48.0,1883-08-12,...,Q1000394,This Modern Age,United States of America,1931-08-29,Q179200,7.0,1924-01-01,United States of America,None,Metro-Goldwyn-Mayer
1,NaN,Q108233,57.0,1900-04-28,Germany,male,Josef Sieber,Q95951,32.0,1925-05-11,...,Q1000174,Tinko,None,1957-01-01,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Q2370079,17.0,1984-11-21,Thailand,male,Suwinit Panjamawat,Q1041384,31.0,1970-09-19,...,Q1000825,Jan Dara,Thailand,2001-01-01,Q716064,39.0,1962-11-28,People's Republic of China,male,Peter Chan
3,NaN,Q1269500,37.0,1939-05-10,Croatia,male,Ivica Vidović,Q603685,46.0,1930-02-14,...,Q1001102,The Rat Savior,Socialist Federal Republic of Yugoslavia,1976-10-26,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Q298818,44.0,1925-02-18,United States of America,male,George Kennedy,Q313040,48.0,1921-10-01,...,Q1000826,Guns of the Magnificent Seven,United States of America,1969-01-01,Q3559718,49.0,1920-07-05,United States of America,male,Vincent M. Fennelly


Reorder and rename the columns

In [24]:
cols_reordered = [
    'movie_id', 'movie_name', 'movie_genre', 'movie_origin', 'movie_release', 'box_office_value',
    'director_id', 'director_id_name', 'director_id_gender', 'director_id_country', 'director_id_birth', 'director_id_age_at_release',
    'producer_id', 'producer_id_name', 'producer_id_gender', 'producer_id_country', 'producer_id_birth', 'producer_id_age_at_release',
    'cast_id_0', 'cast_id_0_name', 'cast_id_0_gender', 'cast_id_0_country', 'cast_id_0_birth', 'cast_id_0_age_at_release',
    'cast_id_1', 'cast_id_1_name', 'cast_id_1_gender', 'cast_id_1_country', 'cast_id_1_birth', 'cast_id_1_age_at_release',
    'cast_id_2', 'cast_id_2_name', 'cast_id_2_gender', 'cast_id_2_country', 'cast_id_2_birth', 'cast_id_2_age_at_release',
    'cast_id_3', 'cast_id_3_name', 'cast_id_3_gender', 'cast_id_3_country', 'cast_id_3_birth', 'cast_id_3_age_at_release',
    'cast_id_4', 'cast_id_4_name', 'cast_id_4_gender', 'cast_id_4_country', 'cast_id_4_birth', 'cast_id_4_age_at_release',
]
df_chars_extended = df_chars_extended[cols_reordered]
df_chars_extended.head()

,movie_id,movie_name,movie_genre,movie_origin,movie_release,box_office_value,director_id,director_id_name,director_id_gender,director_id_country,...,cast_id_3_gender,cast_id_3_country,cast_id_3_birth,cast_id_3_age_at_release,cast_id_4,cast_id_4_name,cast_id_4_gender,cast_id_4_country,cast_id_4_birth,cast_id_4_age_at_release
0,Q1000394,This Modern Age,drama film,United States of America,1931-08-29,NaN,Q1985392,Nick Grinde,male,United States of America,...,female,France,1878-06-02,53.0,Q2831060,Albert Conti,male,United States of America,1887-01-29,44.0
1,Q1000174,Tinko,None,None,1957-01-01,NaN,Q1379822,Herbert Ballmann,male,Germany,...,male,Germany,1898-02-21,59.0,Q1442953,Hans Hardt-Hardtloff,male,Germany,1906-08-08,51.0
2,Q1000825,Jan Dara,LGBT-related film,Thailand,2001-01-01,NaN,Q360509,Nonzee Nimibutr,male,Thailand,...,female,Thailand,1940-01-24,61.0,Q16305292,NaN,male,None,1975-03-26,26.0
3,Q1001102,The Rat Savior,horror film,Socialist Federal Republic of Yugoslavia,1976-10-26,NaN,Q3199956,Krsto Papić,male,Croatia,...,male,Croatia,1932-01-04,44.0,Q12628320,Branko Špoljar,male,None,1914-01-05,62.0
4,Q1000826,Guns of the Magnificent Seven,Western film,United States of America,1969-01-01,NaN,Q450601,Paul Wendkos,male,United States of America,...,male,United States of America,1914-07-24,55.0,Q465643,George Rigaud,male,Argentina,1905-08-11,64.0


In [25]:
cols_renamed = [
    'movie_id', 'movie_name', 'movie_genre', 'movie_origin', 'movie_release', 'box_office_value',
    'director_id', 'director_name', 'director_gender', 'director_country', 'director_birth', 'director_age_at_release',
    'producer_id', 'producer_name', 'producer_gender', 'producer_country', 'producer_birth', 'producer_age_at_release',
    'cast_id_0', 'cast_0_name', 'cast_0_gender', 'cast_0_country', 'cast_0_birth', 'cast_0_age_at_release',
    'cast_id_1', 'cast_1_name', 'cast_1_gender', 'cast_1_country', 'cast_1_birth', 'cast_1_age_at_release',
    'cast_id_2', 'cast_2_name', 'cast_2_gender', 'cast_2_country', 'cast_2_birth', 'cast_2_age_at_release',
    'cast_id_3', 'cast_3_name', 'cast_3_gender', 'cast_3_country', 'cast_3_birth', 'cast_3_age_at_release',
    'cast_id_4', 'cast_4_name', 'cast_4_gender', 'cast_4_country', 'cast_4_birth', 'cast_4_age_at_release',
]
df_chars_extended.columns = cols_renamed
df_chars_extended.head()

,movie_id,movie_name,movie_genre,movie_origin,movie_release,box_office_value,director_id,director_name,director_gender,director_country,...,cast_3_gender,cast_3_country,cast_3_birth,cast_3_age_at_release,cast_id_4,cast_4_name,cast_4_gender,cast_4_country,cast_4_birth,cast_4_age_at_release
0,Q1000394,This Modern Age,drama film,United States of America,1931-08-29,NaN,Q1985392,Nick Grinde,male,United States of America,...,female,France,1878-06-02,53.0,Q2831060,Albert Conti,male,United States of America,1887-01-29,44.0
1,Q1000174,Tinko,None,None,1957-01-01,NaN,Q1379822,Herbert Ballmann,male,Germany,...,male,Germany,1898-02-21,59.0,Q1442953,Hans Hardt-Hardtloff,male,Germany,1906-08-08,51.0
2,Q1000825,Jan Dara,LGBT-related film,Thailand,2001-01-01,NaN,Q360509,Nonzee Nimibutr,male,Thailand,...,female,Thailand,1940-01-24,61.0,Q16305292,NaN,male,None,1975-03-26,26.0
3,Q1001102,The Rat Savior,horror film,Socialist Federal Republic of Yugoslavia,1976-10-26,NaN,Q3199956,Krsto Papić,male,Croatia,...,male,Croatia,1932-01-04,44.0,Q12628320,Branko Špoljar,male,None,1914-01-05,62.0
4,Q1000826,Guns of the Magnificent Seven,Western film,United States of America,1969-01-01,NaN,Q450601,Paul Wendkos,male,United States of America,...,male,United States of America,1914-07-24,55.0,Q465643,George Rigaud,male,Argentina,1905-08-11,64.0


Save the dataframe to the processed data folder

In [26]:
df_chars_extended.to_csv(os.path.join(PROCESSED_DATA_PATH, 'chars_extended.csv'), index=False)

In [11]:
df_chars_extended = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, 'chars_extended.csv'))